In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import warnings

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

# Function to get a synonym of a word
def get_synonym(word):
    synonyms = nltk.corpus.wordnet.synsets(word)
    if synonyms:
        return random.choice(synonyms[0].lemma_names())
    else:
        return word

# Function to augment text data
def augment_text(text, n=1):
    augmented_texts = []
    words = nltk.word_tokenize(text)

    for _ in range(n):
        augmented_text = words.copy()

        # Random Deletion
        augmented_text = [word for word in augmented_text if random.uniform(0, 1) > 0.2]

        # Random Swap
        for _ in range(2):
            if len(augmented_text) >= 2:
                idx1, idx2 = random.sample(range(len(augmented_text)), 2)
                augmented_text[idx1], augmented_text[idx2] = augmented_text[idx2], augmented_text[idx1]

        # Random Insertion
        for _ in range(2):
            if augmented_text:
                idx = random.randint(0, len(augmented_text) - 1)
                word = get_synonym(augmented_text[idx])
                augmented_text.insert(idx, word)

        augmented_texts.append(' '.join(augmented_text))

    return augmented_texts

# Load the dataset
dataset_path = r'All questions answers of Stack Exchange.csv'
df = pd.read_csv(dataset_path, encoding='latin1')

# Select relevant columns for classification
selected_columns = ['QuestionTitle', 'QuestionBody', 'Negotiation']
df = df[selected_columns]

# Clean and preprocess text data
df['text'] = df['QuestionTitle'] + ' ' + df['QuestionBody'] 

# Convert to lowercase
df['text'] = df['text'].str.lower()

# Remove punctuation
df['text'] = df['text'].astype(str).apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Remove stopwords
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Lemmatization
lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

# Data augmentation using the augment_text function
augmented_texts = []
for text in df['text']:
    augmented_texts.extend(augment_text(text, n=1))

# Combine original and augmented texts
df_augmented = pd.DataFrame({'text': df['text'].tolist() + augmented_texts, 'Negotiation': df['Negotiation'].tolist() + [neg for neg in df['Negotiation'].tolist() for _ in range(1)]})
df_augmented.dropna(inplace=True)

# Get unique classes and clean negotiation names
df_augmented['Negotiation'] = df_augmented['Negotiation'].str.strip().str.lower()
unique_negotiations_augmented = df_augmented['Negotiation'].unique()

# Split the augmented data into training and testing sets
X_train_augmented, X_test_augmented, y_train_augmented, y_test_augmented = train_test_split(df_augmented['text'], df_augmented['Negotiation'], test_size=0.2, random_state=42)

# Define maximum number of words to consider as features
max_features = 5000

# Define maximum length of a sequence
max_sequence_length = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_augmented['text'])

X_train_augmented = tokenizer.texts_to_sequences(X_train_augmented)
X_test_augmented = tokenizer.texts_to_sequences(X_test_augmented)

X_train_augmented = pad_sequences(X_train_augmented, maxlen=max_sequence_length)
X_test_augmented = pad_sequences(X_test_augmented, maxlen=max_sequence_length)

encoder_augmented = LabelEncoder()
y_train_encoded_augmented = encoder_augmented.fit_transform(y_train_augmented)
y_test_encoded_augmented = encoder_augmented.transform(y_test_augmented)

oversample_augmented = RandomOverSampler(sampling_strategy='minority')
X_over_augmented, y_over_augmented = oversample_augmented.fit_resample(X_train_augmented, y_train_encoded_augmented)

y_train_one_hot_augmented = to_categorical(y_over_augmented)
y_test_one_hot_augmented = to_categorical(y_test_encoded_augmented)

# Define the RNN model for augmented data
model_augmented = Sequential()

# Embedding layer
model_augmented.add(Embedding(input_dim=max_features, output_dim=512, input_length=max_sequence_length))

# RNN Layer
model_augmented.add(SimpleRNN(128, return_sequences=True))

# Global Max Pooling Layer
model_augmented.add(GlobalMaxPooling1D())

# Fully Connected Layer
model_augmented.add(Dense(64, activation='relu'))

# Output Layer
model_augmented.add(Dense(len(unique_negotiations_augmented), activation='softmax'))

# Compile the model with the best learning rate for augmented data
model_augmented.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])  

# Train the model with oversampled augmented data for the best number of epochs
model_augmented.fit(X_over_augmented, y_train_one_hot_augmented, epochs=50, batch_size=64, verbose=0) 

# Make predictions on the augmented test set
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    predictions_augmented = model_augmented.predict(X_test_augmented)

# Convert predictions from one-hot vectors to labels for augmented data
predictions_augmented = encoder_augmented.inverse_transform(predictions_augmented.argmax(axis=1))


# Print Grid Search Parameters used in the RNN model 
print("\nParameters used in the RNN model:")
print(f"- Embedding Dimension: 512")
print(f"- Learning Rate: 0.01")
print(f"- Batch Size: 64")
print(f"- Number of Epochs: 50")
print()

# Evaluate the performance of the classifier with augmented data
accuracy_augmented = accuracy_score(y_test_augmented, predictions_augmented)
print(f'Best Model Accuracy using RNN: {accuracy_augmented:.2f}')


# Display classification report for augmented data
print('\nClassification Report for Best Model with Augmented Data using RNN:\n', classification_report(y_test_augmented, predictions_augmented, labels=unique_negotiations_augmented, zero_division=1))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nekdilkhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nekdilkhan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nekdilkhan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 172ms/step

Parameters used in the RNN model:
- Embedding Dimension: 512
- Learning Rate: 0.01
- Batch Size: 64
- Number of Epochs: 50

Best Model Accuracy using RNN: 0.62

Classification Report for Best Model with Augmented Data using RNN:
               precision    recall  f1-score   support

  conceptual       0.54      0.68      0.60       260
 theoretical       0.56      0.31      0.40       161
    learning       0.58      0.52      0.55        61
     tooling       0.57      0.63      0.60       233
      errors       0.79      0.78      0.79       336
   api usage       0.43      0.40      0.41        81

    accuracy                           0.62      1132
   macro avg       0.58      0.55      0.56      1132
weighted avg       0.62      0.62      0.61      1132

